# 수리적 머신러닝 분류화 과제_25.3.17
## 120240106 수학과 김건휘

## 사용할 Dataset : Wine
**Dataset 출처** : https://quadram.ac.uk/ , https://www.timeseriesclassification.com/description.php?Dataset=Wine

**Dataset Description** : 	Food spectrographs of two types of wine. The classes are Cabernet Sauvignon' and 'Shiraz'. Obtained using Fourier transform infrared (FTIR) spectroscopy with attenuated total reflectance (ATR) sampling.

**Dataset 구조 설명** :
첫 번째 열은 Class로 1과 2
2 ~ 235 열은 해당 Wine의 feature

    **234개의 feature란?**
    - FTIR 장비는 특정한 주파수 범위(예: 4000cm⁻¹ ~ 400cm⁻¹)에서 적외선을 쏨
    - 각각의 주파수에서 흡광도(Absorbance)를 측정 → 이 값들이 특징(feature) 벡터가 됨
    - 234개의 데이터 포인트는 234개의 서로 다른 주파수에서 측정된 흡광도 값을 의미

## 데이터 전처리

### Dataset 불러오기

위 주소에서 Dataset 다운로드 후, Colab 파일에 업로드

In [ ]:
# 압축 폴더 이름 설정
img_folder_nm = 'Wine'

# 다운로드된 압축 파일 이름
img_zipfile_nm = 'Wine.zip'

In [ ]:
# 압축 풀기
%%capture
!unzip {img_zipfile_nm} -d {img_folder_nm}

### Dataset 합치기
만약 Train Dataset과 Test Dataset이 구분되어 있었다면, 원본 데이터로 만들어주는 작업

In [ ]:
from pathlib import Path

In [ ]:
train_csv = Path('/content/Wine/Wine_TRAIN.txt')
valid_csv = Path('/content/Wine/Wine_TEST.txt')

In [ ]:
import pandas as pd

In [ ]:
train_df = pd.read_csv(train_csv, delim_whitespace=True, header=None)
train_df

In [ ]:
valid_df = pd.read_csv(valid_csv, delim_whitespace=True, header=None)
valid_df

In [ ]:
# concatenate train and valid set together

df = pd.concat([train_df, valid_df])
df.shape

### Label, Feature 분리

In [ ]:
X = df.iloc[:, 1:]  # 특징 데이터
y = df.iloc[:, 0]   # 클래스 라벨

### Train/Test 분할

예제 코드처럼 60퍼센트로 설정 시, Class가 1 인 데이터들이 지배적으로 들어가게 됨.
(위의 데이터프레임 구조 df 확인)

In [ ]:
# n_samples = len(y)
# train_size = int(n_samples * 0.6)  # 60%를 학습 데이터로 사용

클래스 비율을 적절히 뽑을 수 있는 stratify를 이용

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.4,         # 테스트 세트 40%
    random_state=42,       # 재현성을 위한 난수 고정
    shuffle=True,          # 데이터를 섞어서 분할
    stratify=y             # 클래스 비율 동일하게 유지
)

In [ ]:
X_train

In [ ]:
y_train

## 데이터 정규화 작업

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Label Encoder 초기화
le = LabelEncoder()

In [ ]:
y_train = train_df[[0]].to_numpy(dtype=int).reshape(-1)
y_valid = valid_df[[0]].to_numpy(dtype=int).reshape(-1)

In [ ]:
# label encoder (e.g, change ['bear', 'rabbit', 'dog'] to [0,1,2])

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_train_new = le.fit_transform(y_train)
y_valid_new = le.transform(y_valid)

y_train_new

In [ ]:
y_train

In [ ]:
# from new to old label
le.inverse_transform([0, 1])

In [ ]:
# scale using standard score

from sklearn.preprocessing import StandardScaler

# load scaler
# 표준화를 통해 데이터의 평균을 0, 표준 편차를 1로 조정하여 데이터를 정규 분포 형태로
scaler = StandardScaler()

# scale train and valid sets

# transform dataframes (take out first column which is label)
X_train = pd.DataFrame(scaler.fit_transform(train_df.iloc[:, 1:]), dtype='float32')
X_valid = pd.DataFrame(scaler.transform(valid_df.iloc[:, 1:]), dtype='float32')

## XGboost을 이용한 분류

In [ ]:
import xgboost as xgb
print(xgb.__version__)

In [ ]:
# set XGBoost regressor parameters
my_random_seed = 128
early_stop_rounds = 20

early_stop = xgb.callback.EarlyStopping(rounds=early_stop_rounds, save_best=True)

xgb_classify = xgb.XGBClassifier(random_state=my_random_seed, callbacks=[early_stop])

In [ ]:
%%time

## train

# fit
xgb_classify.fit(X_train, y_train_new,
                 eval_set=[(X_valid, y_valid_new)], verbose=True)

In [ ]:
# predict

y_predicted_vaild = xgb_classify.predict(X_valid)
y_predicted_vaild

### 성능 평가

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [ ]:
# 성능 평가
print("Accuracy:", accuracy_score(y_valid_new, y_predicted_vaild))
print("Confusion Matrix:\n", confusion_matrix(y_valid_new, y_predicted_vaild))

In [ ]:
# pos_label을 1.0으로 설정
print("Precision (Class=1.0):", precision_score(y_valid_new, y_predicted_vaild, pos_label=1.0))
print("Recall (Class=1.0):", recall_score(y_valid_new, y_predicted_vaild, pos_label=1.0))
print("F1-Score (Class=1.0):", f1_score(y_valid_new, y_predicted_vaild, pos_label=1.0))

## XGboost를 Finetuning 해서 사용하기

In [ ]:
from xgboost import cv
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# range of parameters to do search

param_grid = {
    "max_depth": [3, 4, 5, 6],
    "learning_rate": [0.01, 0.05, 0.1, 0.15],
    "gamma": [0, 0.25, 0.5, 0.75, 1],
    "subsample": [0.4, 0.6, 0.8],
    "colsample_bytree": [0.25, 0.5, 0.75],
    "n_estimators": [100, 150, 200, 250, 300]
}

#     "scale_pos_weight": [1],

In [ ]:
%%time

# Init classifier
xgb_cl = xgb.XGBClassifier(random_state=my_random_seed)

search_seed = 42

# Init Grid Search
search = RandomizedSearchCV(xgb_cl, param_distributions=param_grid, n_jobs=-1, cv=3, verbose=1,
                             random_state=search_seed, return_train_score=True)

# Fit
search.fit(X_train, y_train_new)

In [ ]:
search.best_params_

In [ ]:
search.best_score_

In [ ]:
search.best_estimator_

In [ ]:
early_stop_rounds = 10

xgb_classify_best_estimator = search.best_estimator_.set_params(
    random_state=my_random_seed,
    early_stopping_rounds=early_stop_rounds
)

In [ ]:
xgb_classify_best_estimator

In [ ]:
%%time


xgb_classify_best_estimator.fit(X_train, y_train_new,
            eval_set=[(X_valid, y_valid_new)])

In [ ]:
# predict

y_predicted_vaild_tuned = xgb_classify_best_estimator.predict(X_valid)
y_predicted_vaild_tuned

### 성능 평가

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

In [ ]:
# 성능 평가
print("Accuracy:", accuracy_score(y_valid_new, y_predicted_vaild_tuned))
print("Confusion Matrix:\n", confusion_matrix(y_valid_new, y_predicted_vaild_tuned))

In [ ]:
# pos_label을 1.0으로 설정
print("Precision (Class=1.0):", precision_score(y_valid_new, y_predicted_vaild_tuned, pos_label=1.0))
print("Recall (Class=1.0):", recall_score(y_valid_new, y_predicted_vaild_tuned, pos_label=1.0))
print("F1-Score (Class=1.0):", f1_score(y_valid_new, y_predicted_vaild_tuned, pos_label=1.0))